In [2]:
# install octave
!sudo apt-get -qq update
!sudo apt-get -qq install octave octave-signal liboctave-dev

# install oct2py that compatible with colab
import os

from pkg_resources import get_distribution

os.system(
    f"pip install -qq"
    f" ipykernel=={get_distribution('ipykernel').version}"
    f" ipython=={get_distribution('ipython').version}"
    f" tornado=={get_distribution('tornado').version}"
    f" oct2py"
)

# install packages
!pip install -qq matpower matpowercaseframes

'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.
C:\Users\Naufal A'\AppData\Local\Temp\ipykernel_31092\4024551122.py:8: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import get_distribution


In [3]:
import matpower
import oct2py

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.8.0
matpower version: 8.0.0.2.2.2


In [4]:
# import start_instance to start matpower instance
from matpower import start_instance

# start instance
m = start_instance()

m.eval(
    """
    mpopt = mpoption('verbose', 2);
    mpc = loadcase('case14');
    _r1 = runopf(mpc, mpopt);
    """
)

# fech data to python (.eval is used because .pull is not working in acessing field)
r1_mpc = m.eval(
    "struct("
    " 'baseMVA', _r1.baseMVA, 'version', _r1.version, 'bus', _r1.bus, 'gen', _r1.gen,"
    " 'branch', _r1.branch, 'gencost', _r1.gencost);"
)


MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0     13212.573                0.624343        0.275      18.5289            0
  1     8077.4637    0.77403    0.0237554    0.0669394      6.14623     0.221221
  2     8332.0151    0.14646   5.5179e-05    0.0423581      1.61662    0.0140811
  3     8272.2352   0.095342  6.05388e-05     0.058572     0.193778   0.00326095
  4     8132.6028    0.38603  0.000981254   0.00654534    0.0253149   0.00764178
  5     8108.5602     0.1963  0.000743622   0.00106529   0.00469505   0.00132593
  6     8086.8978    0.13748  0.000934896   0.00124858   0.00124516   0.00119625
  7     8083.3486   0.085386  0.000153596   0.00183126  0.000400753  0.000196232
  8     8

In [5]:
[GEN_BUS, PG, QG, QMAX, QMIN, VG, MBASE, GEN_STATUS, PMAX, PMIN, MU_PMAX, 
 MU_PMIN, MU_QMAX, MU_QMIN, PC1, PC2, QC1MIN, QC1MAX, QC2MIN, QC2MAX, 
 RAMP_AGC, RAMP_10, RAMP_30, RAMP_Q, APF] = m.idx_gen(nout='max_nout')
gen_index = 5 # index of generator to be changed
gen_index_ = int(gen_index - 5) # -1 due to python indexing start from 0
PMAX_ = int(PMAX - 1)
r1_mpc['gen'][gen_index_, PMAX_]         # max capacity before change

332.4

In [6]:

r1_mpc['gen'][gen_index_, PMAX_] *= 0.5  # reduce capacity by 50%
r1_mpc['gen'][gen_index_, PMAX_]        # max capacity after change

166.2

In [7]:
#re-run power flow with modified generator
# push back value to octave client
m.push('mpc', r1_mpc) # push r1_mpc in python to mpc in octave

# test if we can retrive pushed value
mpc = m.pull('mpc')

# test if our pushed variable can be used
m.eval("_r1 = runopf(mpc, mpopt);")


MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0     10659.143                  0.4155        0.275           24            0
  1     8747.5364    0.39122    0.0174348    0.0206894      7.34549    0.0925308
  2     8837.1545   0.096549  0.000106025    0.0187991      1.46238   0.00478026
  3     8610.0301    0.19238  0.000260844    0.0612392     0.307321    0.0120573
  4     8276.4655    0.45147    0.0014423    0.0201797    0.0558664    0.0179239
  5      8171.825    0.27371  0.000859772   0.00116517   0.00715737   0.00572542
  6     8136.4296    0.14626    0.0009201   0.00220213   0.00188878   0.00194782
  7     8129.2025   0.080752   0.00053876   0.00299006  0.000420266  0.000398484
  8     8

In [11]:
pip install nb-clean
nb-clean clean notebooks/ -o

SyntaxError: invalid syntax (3143525369.py, line 1)